CNN LSTM 

Data Set 14
Using data zero data zero padding


In [1]:
%load_ext tensorboard
import tensorflow as tf
import datetime

import numpy as np
import pandas as pd
from keras.models import Sequential, Model, load_model
from keras.layers import Input ,Dense, Dropout, Activation, LSTM, Conv1D
from keras.layers import Lambda, Convolution1D, MaxPooling1D, Flatten, Reshape, BatchNormalization
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.metrics import categorical_crossentropy, binary_crossentropy

#from spp.SpatialPyramidPooling import SpatialPyramidPooling

#For data saving
import pickle
import random
#other imports
import gzip
import glob
import os
import keras.backend as K
import os
import time 
#cwd = os.path.dirname(os.path.realpath("SURF_001_TwoClass13.ipynb"))

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

!rm -rf ./logs/ 

In [2]:
genome = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

**Creating Dataframe**

https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

In [ ]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(1000, 4), n_channels=0,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, 1000, 4))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('../model_11_data_4/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [4]:
dense_layer = 1
layer_size = 64 
conv_layer = 2
seq_length = 1000
base_pair = 4
num_strides = 1
pool_size = 4

In [ ]:
test_count = 0
with open('../partition_train_val_a.dat', 'rb') as f:
    partition = pickle.load(f)
for index in partition['train']:
    a = partition['labels'][index][0][0]
    if test_count < 10:
        print(a)
    partition['labels'][index] = a
    test_count += 1
    if test_count < 10:
        print(partition['labels'][index])
for index in partition['validation']:
    a = partition['labels'][index][0][0]
    partition['labels'][index] = a
    test_count += 1
    if test_count < 20:
        print(partition['labels'][index])


In [ ]:
cat_1 = 0
cat_0 = 0
for i in partition['train']:
    if partition['labels'][i] == 0:
        cat_0 += 1
    elif partition['labels'][i] == 1:
        cat_1 += 1
    else:
        print('issue')

In [ ]:
print(cat_1)
print(cat_0)

In [ ]:
# Parameters
params = {'dim': (32,1000,4),
          'batch_size': 64,
          'n_classes': 2,
          'n_channels': 1,
          'shuffle': True}

# Datasets
#with open('../partition_train_val_a.dat', 'rb') as f:
   # partition = pickle.load(f)

labels = partition['labels']

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model = Sequential()
model.add(Conv1D(filters=layer_size, kernel_size=(4), input_shape=(seq_length, 4), activation='relu', strides=1, padding='valid'))
model.add(Conv1D(layer_size, 4, batch_input_shape=(32, seq_length, 4), strides=1, padding='valid', activation='relu'))
model.add(MaxPooling1D(pool_size=4, strides=None, padding='valid', data_format='channels_last'))
model.add(Dropout(0.1, noise_shape=None, seed=None))

model.add(Flatten())

model.add(Dense(12))
model.add(Activation('relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False)

In [9]:
with open('../partition_train_val_a.dat', 'rb') as f:
    partition = pickle.load(f)
train_samples = partition['train']
val_samples = partition['validation']
random.shuffle(train_samples)
random.shuffle(val_samples)

partition['train'] = train_samples
partition['validation'] = val_samples

'''
a = partition['train'][0]
print(a)
print(partition['labels'][a] )
train_labels = np.empty(1, dtype=np.int8)
train_samples = []
train_labels[0] = partition['labels'][a] 
train_samples = np.load('../model_11_data_4/' + partition['train'][0] + '.npy')


print(train_labels.shape)
print(train_samples.shape)


val_labels = train_labels
val_samples = train_samples
print(val_labels.shape)
print(val_samples.shape)
'''



# Parameters
params = {'dim': (32,1000,4),
          'batch_size': 64,
          'n_classes': 2,
          'n_channels': 1,
          'shuffle': True}

# Datasets
#with open('../partition_train_val_a.dat', 'rb') as f:
   # partition = pickle.load(f)

labels = partition['labels']

# Generators
#training_generator = DataGenerator(partition['train'], labels, **params)
#validation_generator = DataGenerator(partition['validation'], labels, **params)

dense_layers = [1]
layer_sizes = [64, 128]
conv_layers = [1, 2]

# Design model
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            Earlystop = EarlyStopping(
            monitor='val_loss', min_delta=0, patience=25, verbose=0, mode='auto',
            baseline=None, restore_best_weights=True)
            count_input_train = 0
            count_input_validation = 0
            while count_input_train < len(partition['train']):
                Y_train = np.zeros((200, 2), dtype=np.int8)
                X_train = np.zeros((200, 1000, 4), dtype=np.int8)
                Y_val = np.zeros((40, 2), dtype=np.int8)
                X_val = np.zeros((40, 1000, 4), dtype=np.int8)
                if count_input_train < (len(partition['train']) - 200):
                    for i in range(0, 200):
                        a = partition['train'][count_input_train]
                        X_train[i, :, :] = np.load('../model_11_data_4/' + partition['train'][count_input_train] + '.npy') 
                        Y_train[i, :] = partition['labels'][a] 
                        count_input_train += 1
                else:
                    count_input_train += 1000
                if count_input_validation < (len(partition['validation']) - 40):
                    for i in range(0, 40):
                        a = partition['validation'][count_input_validation]
                        X_val[i, :, :] = np.load('../model_11_data_4/' + partition['validation'][count_input_validation] + '.npy') 
                        Y_val[i, :] = partition['labels'][a] 
                        count_input_validation += 1

                model = Sequential()
                model.add(Conv1D(filters=layer_size, kernel_size=(4), input_shape=(seq_length, 4), activation='relu', strides=1, padding='valid'))
                model.add(MaxPooling1D(pool_size=4, strides=None, padding='valid', data_format='channels_last'))
                model.add(Conv1D(layer_size, 4, batch_input_shape=(1, seq_length, 4), strides=1, padding='valid', activation='relu'))
                model.add(MaxPooling1D(pool_size=4, strides=None, padding='valid', data_format='channels_last'))

                model.add(Dropout(0.1, noise_shape=None, seed=None))

                for l in range(conv_layer-1):
                    model.add(MaxPooling1D(pool_size=4, strides=None, padding='valid', data_format='channels_last'))
                    model.add(Conv1D(layer_size, 4, activation='relu'))

                #model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last'))
                #model.add(Conv1D(64, 4, activation='relu'))

                model.add(Dropout(0.1, noise_shape=None, seed=None))

                # model.add(LSTM(32))

                model.add(Flatten())
                
                model.add(Dense(512))
                model.add(Activation("relu"))
                model.add(Dropout(0.1))

                for l in range(dense_layer):
                    model.add(Dense(512))
                    model.add(Activation('relu'))
                model.add(Dense(2, activation='softmax'))

               # optimizer = keras.optimizers.Adam(lr=0.00001)
                model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
                if count_input_train < 400:
                    print(model.summary())
                    model.save('../Models_H5/011_{}-conv-{}-nodes-{}-dense-{}CNNLSTM'.format(conv_layer, layer_size, dense_layer, int(time.time())))
                                # Train model on dataset
                model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=500, callbacks=[Earlystop], verbose=2)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 997, 64)           1088      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 249, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 246, 64)           16448     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 61, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 61, 64)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 61, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 3904)              0

KeyboardInterrupt: 

**OLD CODE BELOW**

**Preparing inputs**

In [ ]:
with open('Version_e/' + str(int(0 / 20000))+'/' + str(int(0 / 100))+'/DataSet_14_train_labels_onehot_batches1_Part_' + str(1) + '.dat', 'rb') as f:
  train_labels = pickle.load(f)
with open('Version_f/' + str(int(0 / 20000))+'/' + str(int(0 / 100))+'/DataSet_14_train_samples_onehot_batches1_Part_' + str(1) + '.dat', 'rb') as f:
  train_samples = pickle.load(f)
print(train_labels.shape)
print(train_samples.shape)

with open('Version_e/' + str(int(0 / 20000))+'/' + str(int(0 / 100))+'/DataSet_14_train_labels_onehot_batches1_Part_' + str(1) + '.dat', 'rb') as f:
  test_labels = pickle.load(f)
with open('Version_f/' + str(int(0 / 20000))+'/' + str(int(0 / 100))+'/DataSet_14_train_samples_onehot_batches1_Part_' + str(1) + '.dat', 'rb') as f:
  test_samples = pickle.load(f)
print(test_labels.shape)
print(test_samples.shape)


'''

with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_train_labels_onehot_Part:' + str(1) + '.dat', 'rb') as f:
  train_labels = pickle.load(f)
with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_train_samples_onehot_Part:' + str(1) + '.dat', 'rb') as f:
  train_samples = pickle.load(f)
'''

In [ ]:
print(train_labels)
print(train_labels.shape)


CNN LSTM Model:

Notes:

Seems to crash when data is stored in batches of 200 and largerst seq length 310400




In [ ]:

# input  images of 4 by 200
model = Sequential()
model.add(Conv1D(filters=layer_size, kernel_size=(4), input_shape=(seq_length, 4), activation='relu', strides=1, padding='valid'))
model.add(Conv1D(layer_size, 4, batch_input_shape=(1, seq_length, 4), strides=1, padding='valid', activation='relu'))
model.add(MaxPooling1D(pool_size=4, strides=None, padding='valid', data_format='channels_last'))

model.add(Dropout(0.1, noise_shape=None, seed=None))

for l in range(conv_layer-1):
  model.add(MaxPooling1D(pool_size=4, strides=None, padding='valid', data_format='channels_last'))
  model.add(Conv1D(layer_size, 4, activation='relu'))

#model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last'))
#model.add(Conv1D(64, 4, activation='relu'))

model.add(Dropout(0.1, noise_shape=None, seed=None))

model.add(LSTM(32))

model.add(Flatten())

for l in range(dense_layer):
  model.add(Dense(12))
  model.add(Activation('relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
model.fit(train_samples, train_labels, validation_data=(test_samples, test_labels), epochs=10)

'''
odel = Sequential()
model.add(Conv1D(filters=layer_size, kernel_size=(4), input_shape=(seq_length, 4), activation='relu', strides=1, padding='valid'))
model.add(MaxPooling1D(pool_size=4, batch_input_shape=(1, seq_length, 4), strides=1, padding='valid', data_format='channels_last'))
model.add(Conv1D(layer_size, 4, strides=1, padding='valid', activation='relu'))
model.add(MaxPooling1D(pool_size=4, strides=1, padding='valid', data_format='channels_last'))
model.add(Dropout(0.1, noise_shape=None, seed=None))

for l in range(conv_layer-1):
  model.add(Conv1D(layer_size, 4, strides=1, padding='valid', activation='relu'))
  model.add(MaxPooling1D(pool_size=4, strides=1, padding='valid', data_format='channels_last'))
  model.add(Dropout(0.1, noise_shape=None, seed=None))

#model.add(LSTM(64))


model.add(Flatten())

#for l in range(dense_layer):
#  model.add(Dense(64))
#  model.add(Activation('relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
#callbacks=[]
model.fit(train_samples, train_labels, validation_data=(test_samples, test_labels), epochs=3)
model.save('gdrive/My Drive/SURF_2020_Weiss/Models_H5/008_DataSet13_{}-conv-{}-nodes-{}-dense-200-epochs-8-version-Part:1-CNNLSTM'.format(conv_layer, layer_size, dense_layer))
#version 3 is with all the chromosomes
'''

In [ ]:
genome = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']
num_place = 2

#1 - 2411
for part in range(3, 54300):
  num_place += 1
  if num_place > 20000:
    num_place -= 20000
  if part != 1:
    with open('gdrive/My Drive/SURF_2020_Weiss/Data_Sets/DataSet13/test5/' + str(int(part / 20000))+'/' + str(int(num_place / 100))+'/DataSet_13_train_labels_onehot_batches5_Part:' + str(part) + '.dat', 'rb') as f:
      train_labels = pickle.load(f)
    with open('gdrive/My Drive/SURF_2020_Weiss/Data_Sets/DataSet13/test5/' + str(int(part / 20000))+'/' + str(int(num_place / 100))+'/DataSet_13_train_samples_onehot_batches5_Part:' + str(part) + '.dat', 'rb') as f:
      train_samples = pickle.load(f)

    shuffle = np.random.permutation(len(train_samples))
    train_samples = train_samples[shuffle]
    train_labels = train_labels[shuffle]
    
    model.fit(train_samples, train_labels, validation_data=(test_samples, test_labels), epochs=3)
    train_samples = []
    train_labels = []
    if part % 100 == 0 or part == 50:
      print(model.summary())
      print('Part:' + str(part))
      model.save('gdrive/My Drive/SURF_2020_Weiss/008_Model/Models_H5/008_DataSet13_{}-conv-{}-nodes-{}-dense-zeroPadding-3-epochs-2-version-Part:{}-CNNLSTM'.format(conv_layer, layer_size, dense_layer, part))
# Versions for Model 008
# Version 1: 1 bat

**Versions:**

Version 5:
  - Issue not loading data
  - Data set 13, data: onehot version with batches of 1, model: batches of 1

Version 6:
 - Got ride of loading, saving, and compiling model each time, ran with data set 13, data: onehot version with batches of 1, model: batches of 1

In [ ]:
%tensorboard --logdir logs
#%tensorboard --logdir logs/fit
#%tensorboard --logdir gdrive/My Drive/SU
#RF_2020_Weiss/Models/E003_CNN_2category

In [ ]:
model.save('gdrive/My Drive/SURF_2020_Weiss/008_Model/Models_H5/008_DataSet13_{}-conv-{}-nodes-{}-dense-zeroPadding-3-epochs-2-version-Part:{}-CNNLSTM'.format(conv_layer, layer_size, dense_layer, part))


Testing the model:


In [ ]:
# Loading testing data: 
genome = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_test_samples_onehot_200bp' + genome[13] + '.dat', 'rb') as f:
  test_samples = pickle.load(f)
  print(test_samples.shape)
with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_test_labels_onehot_200bp' + genome[13] + '.dat', 'rb') as f:
  test_labels = pickle.load(f)
  print(test_labels.shape) 

In [ ]:
# Load a model
model = load_model('gdrive/My Drive/SURF_2020_Weiss/Models_H5/006_DataSet10_2-conv-64-nodes-0-dense-200-epochs-3-version-chrX-CNNLSTM')

In [ ]:
predicted = model.predict_classes(test_samples)
print(len(predicted))
print(predicted[1000:1100])

In [ ]:
expected = []
for label in test_labels:
  for i, num in enumerate(label):
    if num == 1:
      expected.append(i)
count = 0 
count_2 = 0
for x in predicted:
  if x != 0:
    count += 1
for y in expected:
  if y != 0:
    count_2 += 1
print(count)
print(count_2)

In [ ]:
print(expected[1000:1100])

In [ ]:
count = 0
for i, num in enumerate(expected):
  if num == 1 and predicted[i] == 1: 
    count += 1
print(count)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

results_2 = multilabel_confusion_matrix(expected, predicted)
print(results_2)

In [ ]:
testing = [1, 2, 3, 3]
testing_2 = [1, 2, 3 , 3]
results_2 = multilabel_confusion_matrix(testing, testing_2)
print(results_2)

Extra Code:


In [10]:
with open('../partition_test_a.dat', 'rb') as f:
    partition = pickle.load(f)
test_samples_IDs = partition['test']

with open('../test_labels.dat', 'rb') as f:
    test_labels = pickle.load(f)
print(len(test_labels))


16382


In [ ]:
model = load_model('../Models_H5/011_{}-conv-{}-nodes-{}-dense-{}CNNLSTM-9-26-20'.format(1, 64, 0, 1601093561))

In [11]:
predicted = []
count_input_test = 0
num_test_seq = len(partition['test'])
done = False
while count_input_test < num_test_seq and not(done):
    X_test = np.zeros((200, 1000, 4), dtype=np.int8)
    if count_input_test < (num_test_seq - 200):
        for i in range(0, 200):
            a = partition['test'][count_input_test]
            X_test[i, :, :] = np.load('../DataSet_14_Test_2/' + partition['test'][count_input_test] + '.npy') 
            count_input_test += 1
        predict = (model.predict_classes(X_test))
        for label in predict:
            predicted.append(label)
    else:
        done = True
    if count_input_test == 16200:
        done = True
    print(count_input_test)
print(len(predicted))
print(len(test_labels))
print(predicted[0])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16200
16382
1


In [12]:
expected = []
for label in test_labels[0:16200]:
    expected.append(label[0][0])
print(expected[0])


1


In [13]:
# Example of a confusion matrix in Python
from sklearn.metrics import confusion_matrix
 
results = confusion_matrix(expected, predicted)
print(results)
# Printing the precision and recall, among other metrics
#print(metrics.classification_report(y_act, y_pred, labels=["a", 
#"b"]))

[[4232 4336]
 [5585 2047]]


In [ ]:
### HERE

In [ ]:
train_samples.shape

In [ ]:
train_labels.shape
print(train_labels[0][0])
train_labels_flat = []
for i in range(0, len(train_labels)):
  train_labels_flat.append(train_labels[i][0])
print(len(train_labels))

test_labels.shape
print(test_labels[0][0])
test_labels_flat = []
for i in range(0, len(test_labels)):
  test_labels_flat.append(test_labels[i][0])
print(len(test_labels))

In [ ]:
count = 0
for i in test_labels:
  if i[0] == 1:
    count += 1

print(count)

In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_samples, test_samples, train_labels_flat, test_labels_flat

nsamples, nx, ny = X_train.shape
X_train = X_train.reshape((nsamples,nx*ny))

#nsamples, nx, ny = y_train.shape
#y_train = y_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = X_test.shape
X_test = X_test.reshape((nsamples,nx*ny))

#nsamples, nx, ny = y_test.shape
#y_test = y_test.reshape((nsamples,nx*ny))

# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
classifier = svm.SVC(kernel='linear', C=0.01).fit(X_train, y_train)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(classifier, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

In [ ]:
model.save('../Models_H5/011_{}-conv-{}-nodes-{}-dense-{}CNNLSTM-9-26-20'.format(conv_layer, layer_size, dense_layer, int(time.time())))
